
# MobileNet
## Importing required libraries

In [ ]:
from tensorflow.keras.applications import MobileNet

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from keras import models
import time
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from tensorflow.keras.models import load_model
from tqdm import trange
import tensorflow as tf
import pandas as pd
import seaborn as sns 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import cv2
from keras.preprocessing import image
from os import listdir
from os.path import isfile, join
from sklearn.utils import class_weight 

### Building a MobileNet




In [ ]:
img_rows, img_cols = 224, 224 
model = Sequential()
model = MobileNet( include_top = False,  input_shape = (img_rows, img_cols, 3))
for layer in model.layers:
    layer.trainable = True
    
top_model =model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(64,activation='relu')(top_model)
top_model = Dropout(0.5)(top_model)
output = Dense(1,activation='sigmoid')(top_model)
 
model = Model(inputs = model.input, outputs = output)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

### Loading the data



In [ ]:
 
train_data_dir = 'Dataset path/train'
validation_data_dir = 'Dataset path/validation'
test_data_dir = 'Dataset path/test'
 
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
 
batch_size = 128
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='binary')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='binary')

test_generator = validation_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='binary')


# Fetching x_train, y_train variables from train_generator.
train_generator.reset()
x_train, y_train = next(train_generator)
for i in range(len(train_generator)-1): # 
    img, label = next(train_generator)
    x_train = np.append(x_train, img, axis=0 )
    y_train = np.append(y_train, label, axis=0)
 
 
# Fetching x_tvalidation, y_validation variables from validation_generator.    
validation_generator.reset()
x_validation, y_validation = next(validation_generator)
for i in range(len(validation_generator)-1): 
    img, label = next(validation_generator)
    x_validation = np.append(x_validation, img, axis=0 )
    y_validation = np.append(y_validation, label, axis=0)
    
# Fetching x_test, y_test variables from test.    
test_generator.reset()
x_test, y_test = next(test_generator)
for i in range(len(test_generator)-1): 
    img, label = next(test_generator)
    x_test = np.append(x_test, img, axis=0 )
    y_test = np.append(y_test, label, axis=0)
    
nb_train_samples = len(x_train)
nb_validation_samples = len(x_validation) 
nb_test_samples = len(x_test) 
    
class_weight= {0:0.35, 1:0.65}
    
validation_generator.class_indices 

### Training the model


In [ ]:
checkpoint = ModelCheckpoint("./Models/MobileNet.h5",
                             monitor="val_accuracy",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_accuracy', 
                          min_delta = 0, 
                          patience = 20,
                          verbose = 1,
                          restore_best_weights = True)


callbacks = [earlystop, checkpoint]


# Enter the number of training and validation samples here
nb_train_samples = len(x_train)
nb_validation_samples = len(x_test)

epochs = 100 

history = model.fit(
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    class_weight=class_weight
    )